In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np  
import pandas as pd  
from tqdm import tqdm
from datetime import datetime, date, timedelta
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import time
from itertools import product
import datetime as dt
import calendar
import gc
from datetime import date, timedelta

import random

In [2]:
#减少pandas占用的内存
# @from: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/code
# @liscense: Apache 2.0
# @author: weijian
'''函数说明
仅支持object格式和整形，浮点数
入参：
	props:DataFrame
	floattype:浮点数转换类型（所有的浮点数都会被转换到该类型），默认np.float16
	floattoint:是否强制将float转换成int，默认为False
	floatthreshold:float转换成int阈值，大于等于这个阈值则不会转换，默认为0.01
	fillnavalue:缺失值填充值，默认-999
'''
def reduce_mem_usage(props,floattype = np.float16, floattoint = False, floatthreshold = 0.01,fillnavalue = -999):
    # 计算当前内存
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
    # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
    #NAlist = []
    for col in props.columns:
        # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
        if (props[col].dtypes != object):
            # 判断是否是int类型
            isInt = False
            mmax = props[col].max()
            mmin = props[col].min()
            
            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(props[col]).all():
                #NAlist.append(col)
                props[col].fillna(fillnavalue, inplace=True) # 用-999填充
                
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = np.fabs(props[col] - asint)
            result = result.sum()
            if result < floatthreshold: # 绝对误差和小于floatthreshold认为可以转换的，要根据task修改
                if result == 0:
                    isInt = True
                else:
                    isInt = floattoint
            
            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0: # 最小值大于0，转换成无符号整型
                    if mmax <= 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mmax <= 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mmax <= 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else: # 转换成有符号整型
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)  
            else: # 注意：这里对于float都转换成floattype，需要根据你的情况自己更改
                props[col] = props[col].astype(floattype)
            
            # print("dtype after", props[col].dtype)
            # print("********************************")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

In [3]:
train_data = pd.read_csv('C:/ml_data/bd_buyornot/train.csv')
test_data = pd.read_csv('C:/ml_data/bd_buyornot/submission.csv').set_index('customer_id')

In [4]:
train = train_data
test = test_data
train['order_detail_id'] = train['order_detail_id'].astype(np.uint32)
train['order_id'] = train['order_id'].astype(np.uint32)
train['customer_id'] = train['customer_id'].astype(np.uint32)
train['goods_id'] = train['goods_id'].astype(np.uint32)
train['goods_class_id'] = train['goods_class_id'].astype(np.uint32)
train['member_id'] = train['member_id'].astype(np.uint32)
# 处理状态字段，这里同时处理空值，将空值置为0
train['order_status'] = train['order_status'].astype(np.uint8)
train['goods_has_discount'] = train['goods_has_discount'].astype(np.uint8)
train["is_member_actived"].fillna(0, inplace=True)
train["is_member_actived"]=train["is_member_actived"].astype(np.int8)
train["member_status"].fillna(0, inplace=True)
train["member_status"]=train["member_status"].astype(np.int8)
train["customer_gender"].fillna(0, inplace=True)
train["customer_gender"]=train["customer_gender"].astype(np.int8)
train['is_customer_rate'] = train['is_customer_rate'].astype(np.uint8)
train['order_detail_status'] = train['order_detail_status'].astype(np.uint8)
# 处理日期
train['goods_list_time']=pd.to_datetime(train['goods_list_time'],format="%Y-%m-%d")
train['order_pay_time']=pd.to_datetime(train['order_pay_time'],format="%Y-%m-%d")
train['goods_delist_time']=pd.to_datetime(train['goods_delist_time'],format="%Y-%m-%d")
# 检查内存使用
mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
print(u"训练集使用内容 "+ str(mem_train/ 1024**2)+" MB")
print(u"测试集使用内存 "+ str(mem_test/ 1024**2)+" MB")

训练集使用内容 349.8006525039673 MB
测试集使用内存 24.200225830078125 MB


In [5]:
#数值相同的特征直接删除
del train_data['member_status']
del train_data['is_member_actived']

In [6]:
#存在负值的特征order_detail_discount不处理，后续会删除该特征
#goods_price会使用同商品的价格均值填充
temp = train_data.groupby('goods_id')
goods_price_by_id = {}
for i,j in temp:
    temp_prices = j['goods_price'][j['goods_price'] > 0]
    goods_price_by_id[i] = np.mean(temp_prices)
for  i in tqdm(range(len(train_data))):
    if train_data['goods_price'][i] < 0:
        train_data['goods_price'][i] = goods_price_by_id[train_data['goods_id'][i]]

100%|█████████████████████████████████████████████████████████████████████| 2306871/2306871 [02:22<00:00, 16238.97it/s]


In [7]:
#order_detail_status不处理，后续会删除该特征
#order_status会将异常值101,0替换成6
#order_status==1为未付款订单，并不视为正常订单。
train_data['order_status'].replace(101,6,inplace = True)
train_data['order_status'].replace(0,6,inplace = True)
train_data['order_status'].replace(1,0,inplace = True)
train_data['order_status'].replace(6,1,inplace = True)
train_data['order_status'].replace(3,1,inplace = True)
train_data['order_status'].replace(4,1,inplace = True)

In [8]:
#order_detail_amount后续会删除

# 仅保留有用特征

In [9]:
train_data = train_data[['customer_id','customer_gender','order_detail_payment','order_detail_goods_num','is_customer_rate','order_status','goods_list_time','goods_delist_time','order_pay_time']]

In [10]:
#customer_gender应该很有用，但是其缺失值达到了72%，删除；或者构建两个模型，带有customer_gender和不带有customer_gender的
del train_data['customer_gender']

In [11]:
len(train_data)

2306871

In [12]:
#2013年2月是春节，购物狂欢，数据必异常。所以不使用2012-2之前的数据。这里我们可以留个悬念，用两种数据（包含春节和不包含春节的数据）做个对比。
train_data = train_data[train_data['order_pay_time'] >='2013-02-01 00:00:00']

In [13]:
len(train_data)

940655

In [14]:
#修改data时间格式至具体哪一天，小时分钟秒就忽略了。
train_data['date'] = pd.DatetimeIndex(train_data['order_pay_time']).date
del train_data['order_pay_time']

In [15]:
#不花钱的是客户吗？删除没有花钱的数据
train_data.drop(index =train_data[train_data['order_status'] == 0].index, axis=0, inplace = True )
del train_data['order_status']

In [16]:
#每个客户，统计每一天该客户的购买总金额，购买总数量，评价总数量，
df_payment = train_data.groupby(['date','customer_id']).agg({'order_detail_payment': ['sum']})
df_payment.columns = ['day_total_payment']
df_payment.reset_index(inplace=True)
df_payment = df_payment.set_index(
    ["customer_id", "date"])[["day_total_payment"]].unstack(level=-1).fillna(0)
df_payment.columns = df_payment.columns.get_level_values(1)

In [17]:
df_goods = train_data.groupby(['date','customer_id']).agg({'order_detail_goods_num': ['sum']})
df_goods.columns = ['day_total_goods_num']
df_goods.reset_index(inplace=True)
df_goods = df_goods.set_index(
    ["customer_id", "date"])[["day_total_goods_num"]].unstack(level=-1).fillna(0)
df_goods.columns = df_goods.columns.get_level_values(1)

In [18]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [19]:
def prepare_dataset(df_payment, df_goods,t2018, is_train=True):
    day_ranges = [14,30,60,91]
    day_ranges_s = [21,49,84]
    #day_ranges = [7, 14,30,60,91]
    X = {}
    # 整合用户id
    tmp = df_payment.reset_index()
    X['customer_id'] = tmp['customer_id']
    # 消费特征
    print('Preparing payment feature...')
    for i in day_ranges:
        tmp = get_timespan(df_payment, t2018, i, i)
        
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        
        X['max_%s' % i] = tmp.max(axis=1).values
        
        X['sum_%s' % i] = tmp.sum(axis=1).values
    for i in day_ranges:
        tmp = get_timespan(df_payment, t2018 + timedelta(days=-7), i, i)
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
       
        X['max_%s_2' % i] = tmp.max(axis=1).values
        
    for i in day_ranges:
        tmp = get_timespan(df_payment, t2018, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp != 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp != 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp != 0) * np.arange(i, 0, -1)).max(axis=1).values

    # 对此处进行微调，主要考虑近期因素
    for i in range(1, 4):
        X['day_%s_2018' % i] = get_timespan(df_payment, t2018, i*30, 30).sum(axis=1).values
  
    print('Preparing num feature...')
    for i in day_ranges_s:
        tmp = get_timespan(df_goods, t2018, i, i)
            
        X['goods_mean_%s' % i] = tmp.mean(axis=1).values
            
        X['goods_max_%s' % i] = tmp.max(axis=1).values
            
        X['goods_sum_%s' % i] = tmp.sum(axis=1).values
    for i in day_ranges_s:    
        tmp = get_timespan(df_goods, t2018 + timedelta(weeks=-1), i, i)

        X['goods_mean_%s_2' % i] = tmp.mean(axis=1).values

        X['goods_max_%s_2' % i] = tmp.max(axis=1).values
        X['goods_sum_%s_2' % i] = tmp.sum(axis=1).values
    for i in day_ranges_s:    
        tmp = get_timespan(df_goods, t2018, i, i)
        X['goods_has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['goods_last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['goods_first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values


    # 对此处进行微调，主要考虑近期因素
    for i in range(1, 4):
        X['goods_day_%s_2018' % i] = get_timespan(df_goods, t2018, i*28, 28).sum(axis=1).values


    X = pd.DataFrame(X)
    
    reduce_mem_usage(X)
    
    if is_train:
        # 这样转换之后，打标签直接用numpy切片就可以了
        # 当然这里前提是确认付款总额没有负数的问题
        X['label'] = df_goods[pd.date_range(t2018, periods=30)].max(axis=1).values
        X['label'][X['label'] > 0] = 1
        return X
    return X

In [20]:
num_days = 4
t2017 = date(2013, 7, 1)
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    
    X_tmp = prepare_dataset(df_payment, df_goods,t2017 + delta)
    
    X_tmp = pd.concat([X_tmp], axis=1)

    X_l.append(X_tmp)
    
X_train = pd.concat(X_l, axis=0)
del X_l, y_l

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 344.4864196777344 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07293701171875  MB
This is  21.212138661395763 % of the initial size


 25%|████████████████████▊                                                              | 1/4 [02:00<06:01, 120.38s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 344.4864196777344 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07293701171875  MB
This is  21.212138661395763 % of the initial size


 50%|█████████████████████████████████████████▌                                         | 2/4 [03:54<03:57, 118.57s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 344.4864196777344 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07293701171875  MB
This is  21.212138661395763 % of the initial size


 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [06:09<02:03, 123.39s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 344.4864196777344 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.72537326812744  MB
This is  21.40153255884433 % of the initial size


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [08:24<00:00, 126.20s/it]


In [54]:
num_days = 4
t2017 = date(2013, 7, 1)
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    
    X_tmp = prepare_dataset(df_payment, df_goods,t2017 + delta)
    
    X_tmp = pd.concat([X_tmp], axis=1)

    X_l.append(X_tmp)
    
X_train = pd.concat(X_l, axis=0)
del X_l, y_l

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 307.95003509521484 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07298278808594  MB
This is  23.728843792952503 % of the initial size


 25%|█████████████████████                                                               | 1/4 [00:33<01:40, 33.65s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 307.95003509521484 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07298278808594  MB
This is  23.728843792952503 % of the initial size


 50%|██████████████████████████████████████████                                          | 2/4 [01:06<01:07, 33.55s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 307.95003509521484 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.07298278808594  MB
This is  23.728843792952503 % of the initial size


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [01:40<00:33, 33.49s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 307.95003509521484 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.72541904449463  MB
This is  23.940708115749857 % of the initial size


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:13<00:00, 33.44s/it]


In [ ]:
num_days = 4
t2017 = date(2013, 6, 1)
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    
    X_tmp = prepare_dataset(df_payment, df_goods,t2017 + delta)
    
    X_tmp = pd.concat([X_tmp], axis=1)

    X_l.append(X_tmp)
    
X_train = pd.concat(X_l, axis=0)
del X_l, y_l

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 707.3564300537109 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  166.34874534606934  MB
This is  23.516962351418528 % of the initial size


 25%|█████████████████████                                                               | 1/4 [01:06<03:18, 66.03s/it]

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 707.3564300537109 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  161.85283660888672  MB
This is  22.881369240765498 % of the initial size


 50%|██████████████████████████████████████████                                          | 2/4 [02:21<02:17, 68.92s/it]

Preparing payment feature...


In [20]:
X_train.shape

(6285736, 67)

In [21]:
X_test = prepare_dataset(df_payment, df_goods, date(2013, 9, 1), is_train=False)
X_test = pd.concat([X_test], axis=1)

Preparing payment feature...
Preparing num feature...
Memory usage of the dataframe is : 344.4864196777344 MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  73.72537326812744  MB
This is  21.40153255884433 % of the initial size


In [22]:
X_train.to_csv('C:/ml_data/bd_buyornot/train_data_eda_1.csv')

In [23]:
X_test.to_csv('C:/ml_data/bd_buyornot/test_data_eda_1.csv')